In [ ]:
from __future__ import print_function, division, absolute_import

import matplotlib.pyplot as plt

# import dolfin as dl
# # from hippylib import nb
# from fenics import *

import numpy as np
import logging


logging.getLogger('FFC').setLevel(logging.WARNING)
logging.getLogger('UFL').setLevel(logging.WARNING)
# dl.set_log_active(False)

# np.random.seed(seed=1)

# modify code -----1
# -*- coding: utf-8 -*-

### Import necessary modules and setup the environment


import numpy as np
import sys
import torch
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.distributions import normal
import torch.optim as optim
from scipy import ndimage
from scipy.fftpack import dct, idct
import random
import time
import tracemalloc

from scipy import sparse


import matplotlib.pyplot as plt
# from google.colab import files


from matplotlib import image
from matplotlib import pyplot
from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

from sksparse.cholmod import cholesky
from scipy.sparse import csr_matrix

def mainformula(x,y):
    z = -2* y + 1.6

    # condition1 = (y > .5)
    # z[condition1] = 1

    # condition2 = (x < 0.6) & (x > 0.3) & (y < 0.9) & (y > 0.6)
    # z[condition2] = 0 # np.log(4)

    condition3 = (x<0.4) & (x > 0.1) & (y<0.8) & (y > 0.2)
    m, n = condition3.shape
    condition1 = x < 2
    for i in range(m):
        for j in range(n):
            condition1[i, j] = not condition1[i, j]
            if (not condition3[i, j]) & (i <= m//2):
                condition1[i, j] = True
            condition3[i, j] = not condition3[i, j]
    z[condition3] = 1. # - y + 1.3

    condition2 = (x < 0.9) & (x > 0.45) & (y < 0.9) & (y > 0.6)
    z[condition2] = 0 # np.log(4)
    z[condition1] = 0.5 # np.log(4)


#     condition2 = ( x-.75)** 2 + (y -.75) ** 2 < .1 ** 2
#     z[condition2] = 0.5



    return z

def activation(z):
    return torch.tanh(z)

# activation = nn.ReLU()


M = 100
(x0, xf) = (0, 1.0)
(y0, yf) = (0, 1.0)

hx = (xf - x0)/(M-1)
hy = (yf - y0)/(M-1)

X, Y = np.meshgrid(np.linspace(x0, xf, M), np.linspace(y0, yf, M))
# Z = z_func(X, Y) # evaluation of the function on the grid
Z = mainformula(X, Y).T

c = plt.imshow(Z)
plt.colorbar(c)
plt.title('true image')
plt.show()

def LHS(M):

    N = M**2-1

    main_diag = 4*np.ones((M, 1)).ravel()
    off_diag = -1*np.ones((M-1, 1)).ravel()

    a = main_diag.shape[0]

    diagonals = [main_diag, off_diag, off_diag]

    B = sparse.diags(diagonals, [0,-1,1], shape=(a,a)).toarray()
    B[0, 0] = 3
    B[0,1] = -1.0
    B[M-1,M-2] = -1.0
    B[M-1,M-1] = 3

    D = sparse.diags([-1*np.ones((M+1, 1)).ravel()], [0], shape=(a,a)).toarray()

    C = sparse.diags([-1*np.ones((M+1, 1)).ravel()], [0], shape=(a,a)).toarray()

    e1 = sparse.eye(M).toarray()
    A1 = sparse.kron(e1,B).toarray()

    e2 = sparse.diags([1*np.ones((M, 1)).ravel(),1*np.ones((M, 1)).ravel()], [-1,1], shape=(M,M)).toarray()
    e2[0,1] = 0.0
    e2[M-1,M-2] = 0.0
    A2 = sparse.kron(e2,D).toarray()

    e3 = sparse.diags([1*np.ones((M, 1)).ravel(),1*np.ones((M, 1)).ravel()], [-1,1], shape=(M,M)).toarray()
    e3[1:M-1,0:M] = 0.0
    A3 = sparse.kron(e3,C).toarray()

    mat = A1 + A2 + A3

    for i in range(M):
        mat[i, i] -= 1
        mat[N -i, N -i] -= 1

    return mat

N = M ** 2
N_obs = 2*10**2

#obsInd = np.random.choice((N-1), N_obs, replace = False)
obsStep = np.round(M/(np.sqrt(N_obs)+1)).astype(int)
obsBase = np.arange(obsStep,np.sqrt(N_obs)*obsStep+1,obsStep).astype(int)-1
obsInd = obsBase + (obsStep-1)*M

for p in np.arange(1,np.sqrt(N_obs)).astype(int):
    obsInd = np.append(obsInd,obsBase + (obsStep*p-1)*M)

kappa = 0.01
A = np.identity(M ** 2) + M**2 * kappa * LHS(M)

u_t = np.linalg.solve(A, Z.reshape(-1))

obsX = [index%M for index in obsInd]
obsY = [M-1 - int(index/M) for index in obsInd]


noise_scale = .01

u_obs = u_t.reshape(-1)[obsInd]
u_obs += noise_scale * np.random.normal(0, 1, u_obs.shape)

D_in = 2
H1 = 80
H2 = 80
H3 = 1000
D_out = 1

P = np.zeros((1, N))
for i in obsInd:
    res = np.zeros((1, N));
    res[0, i] = 1
    P = np.concatenate((P, res), axis = 0)


P = P[1:, :]

P_T = P.T
A_T = A.T

factor1 = cholesky(csr_matrix(A))
factor2 = cholesky(csr_matrix(A_T))

m_normal = normal.Normal(0, 1)
class Cauchy(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        Forward equation
        """
        ctx.save_for_backward(input)
        return torch.tan(np.pi*m_normal.cdf(input)-np.pi/2) # input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        Backward equation for auto-computing derivatives
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input = grad_input*((1 + torch.tan(np.pi*m_normal.cdf(input)-np.pi/2).pow(2)))*(np.pi*torch.exp(m_normal.log_prob(input)))
        return grad_input


cauchy = Cauchy.apply

## ============ Customized Neural Network =======
class CauchyNetwork(nn.Module):
    def __init__(self, D_in = 2, H1 = 200, H2 = 60, H3 = 160, D_out = 1):
        super().__init__()
#         torch.manual_seed(1)

        self.w1 = nn.Parameter(torch.randn((H1, D_in),dtype=torch.double))
        self.b1 = nn.Parameter(torch.randn((H1, 1),dtype=torch.double))
        self.w2 = nn.Parameter(torch.randn((H2, H1),dtype=torch.double))
        self.b2 = nn.Parameter(torch.randn((H2, 1),dtype=torch.double))
        self.w3 = nn.Parameter(torch.randn((H3, H2),dtype=torch.double))
        self.b3 = nn.Parameter(torch.randn((H3, 1),dtype=torch.double))
        self.w4 = nn.Parameter(torch.randn((D_out, H3),dtype=torch.double))

    def forward(self, X):
        hidden_1 = activation(self.w1.mm(X) + self.b1)
        hidden_2 = activation(self.w2.mm(hidden_1) + self.b2)
        hidden_3 = activation(self.w3.mm(hidden_2) + self.b3)
        y_pred = self.w4.mm(hidden_3)/list(self.w4.shape)[-1]

#         hidden_1 = torch.tanh(cauchy(self.w1).mm(XX) + cauchy(self.b1))
#         hidden_2 = torch.tanh(cauchy(self.w2).mm(hidden_1) + cauchy(self.b2))
#         hidden_3 = torch.tanh(cauchy(self.w3).mm(hidden_2) + cauchy(self.b3))
#         y_pred = self.w4.mm(hidden_3)/np.sqrt(40)
        return y_pred

class PDEsolver(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, ud):
        """
        Forward equation
        """
        m_numpy = input.detach().numpy().reshape(-1)
        u_numpy = factor1(m_numpy)
        res = P.dot(u_numpy) - ud
        g = factor2(P_T.dot(res))

        ctx.save_for_backward(torch.tensor(g, dtype = torch.double).view(1, -1))

        return torch.tensor(.5 * res.dot(res), dtype = torch.double)


    @staticmethod
    def backward(ctx, grad_output):
        g, = ctx.saved_tensors
        return g, None

XX = torch.tensor(np.concatenate((X.reshape(1, -1), Y.reshape(1, -1)), axis = 0), dtype = torch.double)



m_v1 = []
for ou_i in range(1):
    learning_rate = .01


    # torch.manual_seed(1)
    e = 1
    # gamma = 20

    gm = noise_scale

    reg_para =  gm ** 2



    # net = GaussianNetwork()
    net = CauchyNetwork(D_in = D_in, H1 = H1, H2 = H2, H3 = H3, D_out = D_out)

    solver = PDEsolver.apply



#     loss_fn = torch.nn.MSELoss(reduction='sum', size_average=False)

    adam_steps = 500
    lbfgs_steps = adam_steps+1500

    loss_history = []
    grad_history = []
    reg_history = []
    total_history = []


    opt = torch.optim.Adam(net.parameters(), lr=learning_rate)

    # for k in range(adam_steps):


    optimizer = torch.optim.LBFGS(net.parameters(), lr=learning_rate,line_search_fn='strong_wolfe')


    # print('LBFGS')


    for k in range(lbfgs_steps):
        net.train()
        if k < adam_steps:
            optt = opt
        else:
            optt = optimizer
        if k == adam_steps:
            print('LBFGS')
        def closure():
            optt.zero_grad()
            # loss = neg_log_posterior(net, x_obs_torch, y_obs_torch, std_noise)
            y_pred = net(XX)
            loss = solver(y_pred, u_obs)
#             print(loss)
            loss_l = loss.detach().item()
            reg_loss = 0
            for name, param in net.named_parameters():
                if name == 'w4':
                    reg_loss += param.pow(2).sum()
                else:
                    reg_loss += torch.log(1 + param.pow(2)).sum()

            loss += reg_para * reg_loss
            loss.backward()
            gr = 0
            for param in net.parameters():
                gr += param.grad.pow(2).mean().detach().numpy()/7
    #         for name, param in net.named_parameters():
    #             print(name, param.grad.pow(2).sum())
            loss_history.append( loss_l)
            grad_history.append(gr)
            total_history.append(loss.detach().item())
            reg_history.append((reg_para*reg_loss).detach().item())

            return loss
        # optimizer.zero_grad()
        optt.step(closure)
        if k % 20 == 0:
            print(k)
            y_test = net(XX)
            m_array = y_test.detach().numpy()
            c = plt.imshow(m_array.reshape(M, M))
            plt.show()
            plt.colorbar(c)

#             print(k, loss_l, (reg_para*reg_loss).detach().item(), gr)
    dic2 = {}
    for name, param in net.named_parameters():
        dic2[str(name)] = param.detach()

    hidden_1 = activation(dic2['w1'].mm(XX) + dic2['b1'])
    hidden_2 = activation(dic2['w2'].mm(hidden_1) + dic2['b2'])
    hidden_3 = activation(dic2['w3'].mm(hidden_2) + dic2['b3'])
    y_pred = dic2['w4'].mm(hidden_3)/H3
    m_v1.append(y_pred)

#     hidden_1 = torch.tanh(cauchy(dic1['w1']).mm(XX) + cauchy(dic1['b1']))
#     hidden_2 = torch.tanh(cauchy(dic1['w2']).mm(hidden_1) + cauchy(dic1['b2']))
#     hidden_3 = torch.tanh(cauchy(dic1['w3']).mm(hidden_2) + cauchy(dic1['b3']))
#     y_pred = dic1['w4'].mm(hidden_3)/np.sqrt(40)
#     m_v1.append(y_pred)

    if ou_i == 0:
        zzz = hidden_3.detach().numpy()
    else:
        zzz = np.concatenate((zzz, hidden_3.detach().numpy()))

# c = plt.imshow(m_array.reshape(M, M))
# plt.colorbar(c)

In [ ]:
bs_res = zzz.copy()
nbase = 1000
gmmm = noise_scale
basis = bs_res
Q = P.dot(factor1(basis.T))   #/np.sqrt(nbase)
sigma_Q = np.linalg.inv(1/(gmmm**2) * (Q.T).dot(Q) + 1/1*np.eye(nbase))
MM = sigma_Q.dot(Q.T.dot(u_obs))
Mean_reg = basis.T.dot(MM)/gmmm**2

In [ ]:
plt.rcParams["figure.figsize"] = (5,5)
fig=plt.figure()
ax = plt.axes()

c = plt.imshow(Mean_reg.reshape(M, M), vmin = 0, vmax = 1.2, cmap = 'rainbow')
plt.axis('off')
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(c, cax = cax)
plt.savefig('GRcauchymean.pdf', bbox_inches = "tight")
# plt.clf()